In [1]:
# # test chromedriver
# import time

# from selenium import webdriver



# driver = webdriver.Chrome()  # Optional argument, if not specified will search path.

# driver.get('http://www.google.com/');

# time.sleep(5) # Let the user actually see something!

# search_box = driver.find_element_by_name('q')

# search_box.send_keys('ChromeDriver')

# search_box.submit()

# time.sleep(5) # Let the user actually see something!

# driver.quit()

In [2]:
from googlemaps import GoogleMapsScraper
from datetime import datetime, timedelta
import argparse
import csv
from termcolor import colored
import time

ind = {'most_relevant' : 0, 'newest': 1, 'highest_rating': 2, 'lowest_rating': 3}
HEADER = ['id_review','caption','relative_date','retrieval_date','rating','username','n_review_user','n_photo_user','url_user']
HEADER_W_SOURCE=['id_review','caption','relative_date','retrieval_date','rating','username','n_review_user','n_photo_user','n_photo_user','url_user','url_source']

def csv_writer(source_field, ind_sort_by, path='data/'):
    outfile= ind_sort_by + '_gm_reviews.csv'
    targetfile = open(path+outfile, mode='w', encoding='utf-8',newline='\n')
    writer=csv.writer(targetfile,quoting=csv.QUOTE_MINIMAL)

    if source_field:
        h= HEADER_W_SOURCE
    else:
        h = HEADER
    writer.writerow(h)
    return writer

In [3]:
ind['highest_rating']

2

In [4]:
path = '/Users/akashmeesa/desktop/citydata'
var_to_see_metadata= False
num_args = 20
want_source = False
sort_by= 'highest_rating'

def scrape(filename):

    writer = csv_writer(want_source, sort_by)
    
    with GoogleMapsScraper() as scraper:
        with open(filename,'r') as urls_file:
            for url in urls_file:
                # add way to check if place is false if i want metadata instead of reviews, probably want this. 
                if var_to_see_metadata==True:
                    print(scraper.get_account(url))
                else:
                    print(scraper.get_account(url))
                    error = scraper.sort_by(url, ind[sort_by])
                    if error == 0:
                        n = 0
                        while n< num_args:
                            print('[Review ' + str(n) + ']')
                            reviews = scraper.get_reviews(n)
                            if len(reviews)==0:
                                print('len reviews is 0')
                                break
                            for r in reviews:
                                row_data = list(r.values)
                                if want_source:
                                    row_data.append(url[:-1])
                                writer.writerow(row_data)
                        n += len(reviews)
    return "done"
                                
HEADER
                

['id_review',
 'caption',
 'relative_date',
 'retrieval_date',
 'rating',
 'username',
 'n_review_user',
 'n_photo_user',
 'url_user']

In [5]:
ret = scrape('urls.txt')
ret

{'name': 'Empanada Mama Hell’s Kitchen', 'overall_rating': None, 'n_reviews': 0, 'n_photos': 1720, 'category': 'Latin American restaurant', 'description': 'Stuffed empanadas & arepas top a menu of Latin goodies like plantain chips & guacamole served 24/7.', 'address': None, 'website': None, 'phone_number': None, 'plus_code': None, 'opening_hours': None, 'url': 'https://www.google.com/maps/place/Empanada+Mama+Hell%E2%80%99s+Kitchen/@40.764917,-73.9897431,16.57z/data=!4m6!3m5!1s0x89c25857378598b1:0xe9063cb16f91db4!8m2!3d40.764437!4d-73.9885436!16s%2Fg%2F1vfp93gy?entry=ttu', 'lat': '40.764917', 'long': '-73.9897431'}
[Review 0]
len reviews is 0


'done'